In [2]:
import onnx
from onnx2pytorch import ConvertModel
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

from vnnlib.compat import read_vnnlib_simple

import torch
import numpy as np

from collections import OrderedDict

import csv

import os
from timeit import default_timer as timer

/home/philipp/anaconda3/envs/autoLiRPA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading ONNX and VNNLib Specifications

In [3]:
def load_onnx_model(onnx_path, input_shape):
    onnx_model = onnx.load(onnx_path)
    torch_model = ConvertModel(onnx_model)
    
    x_concrete = torch.zeros(input_shape)
    model = BoundedModule(torch_model, x_concrete)
    return model

In [4]:
def load_vnnlib_spec(vnnlib_path, input_shape, n_out):
    n_in = np.prod(input_shape)
    res = read_vnnlib_simple(vnnlib_path, n_in, n_out)
    bnds, spec = res[0]
    
    bnds = np.array(bnds)
    lbs = bnds[:,0]
    ubs = bnds[:,1]
    
    data_min = torch.tensor(lbs, dtype=torch.float32).reshape(input_shape)
    data_max = torch.tensor(ubs, dtype=torch.float32).reshape(input_shape)
    center = 0.5*(data_min + data_max)

    ptb = PerturbationLpNorm(x_L=data_min, x_U=data_max)
    x = BoundedTensor(center, ptb)
    
    return x

In [5]:
onnx_path = 'example_specs/mnist-net_256x4.onnx'
vnnlib_path = 'example_specs/prop_0_spiral_25.vnnlib'

In [6]:
model = load_onnx_model(onnx_path, [1,1,1,784])
x = load_vnnlib_spec(vnnlib_path, [1,1,1,784], 10)

/home/philipp/anaconda3/envs/autoLiRPA/lib/python3.10/site-packages/onnx2pytorch/convert/layer.py:29: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1654951016690/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/home/philipp/anaconda3/envs/autoLiRPA/lib/python3.10/site-packages/onnx2pytorch/convert/model.py:167: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might no

# Helper Methods

In [7]:
def get_layers(model):
    return [l for l in model.nodes() if l.perturbed]

In [8]:
def get_intermediate_bounds(model):
    """
    Returns a dictionary containing the concrete lower and upper bounds of each layer.
    
    Implemented own method to filter out bounds for weight matrices.
    
    Only call this method after compute_bounds()!
    """
    od = OrderedDict()
    for l in get_layers(model):
        if hasattr(l, 'lower'):
            od[l.name] = (l.lower, l.upper)
            
    return od

# Get Intermediate Bounds

In [9]:
model.compute_bounds(x=(x,), method='ibp')

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3526, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [10]:
bounds_dict = get_intermediate_bounds(model)
for k, (lbs, ubs) in bounds_dict.items():
    print(f"{k}: {lbs.shape}")

/0: torch.Size([1, 1, 1, 784])
/21: torch.Size([1, 784])
/input: torch.Size([1, 256])
/23: torch.Size([1, 256])
/input.3: torch.Size([1, 256])
/25: torch.Size([1, 256])
/input.7: torch.Size([1, 256])
/27: torch.Size([1, 256])
/input.11: torch.Size([1, 256])
/29: torch.Size([1, 256])
/30: torch.Size([1, 10])


In [11]:
bounds_dict['/30']

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3526, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [12]:
model.compute_bounds(x=(x,), method='crown')

(tensor([[-0.4672, -0.4103, -0.4026, -0.3573, -0.4667, -0.4770, -0.3886, -0.4031,
          -0.5523, -0.4308]], grad_fn=<ViewBackward0>),
 tensor([[0.6201, 0.4692, 0.6307, 0.4833, 1.8194, 0.5396, 0.9479, 0.4172, 0.6483,
          0.6467]], grad_fn=<ViewBackward0>))

In [13]:
bounds_dict['/30']

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3526, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [14]:
bounds_dict_crown = get_intermediate_bounds(model)
bounds_dict_crown['/30']

(tensor([[-0.4672, -0.4103, -0.4026, -0.3573, -0.4667, -0.4770, -0.3886, -0.4031,
          -0.5523, -0.4308]], grad_fn=<ViewBackward0>),
 tensor([[0.6201, 0.4692, 0.6307, 0.4833, 1.8194, 0.5396, 0.9479, 0.4172, 0.6483,
          0.6467]], grad_fn=<ViewBackward0>))

**Attention**: CROWN-bounds are only saved for pre-activation nodes and the output!
(in contrast to interval propagation bounds, that are saved for every layer)

In [15]:
bounds_dict_crown.keys()

odict_keys(['/0', '/input', '/input.3', '/input.7', '/input.11', '/30'])

In [16]:
lbs11_ibp, ubs11_ibp = bounds_dict['/input.11']
lbs11_crown, ubs11_crown = bounds_dict_crown['/input.11']

print(lbs11_ibp[:,:10])
print(lbs11_crown[:,:10])

tensor([[-171.4623, -160.4540, -304.7760, -128.2996, -153.7493, -117.9111,
         -123.5061, -278.8083, -158.6687, -207.4034]],
       grad_fn=<SliceBackward0>)
tensor([[-1.8260, -3.1913, -2.1607, -2.8250, -4.0157, -2.1657, -0.7982, -1.6259,
         -1.0763, -5.0089]], grad_fn=<SliceBackward0>)


# Sampling via CROWN

In order to use CROWN to calculate bounds for the sampled directions, we make use of the possibility to supply
- a constraint matrix (which we use to represent the sampled directions) and
- to specify the output layer (which we just set to the layer, for which we want to sample)

The shape of the constraint matrix is `(batch, n_directions, n_neurons)`, where we just set `batch=1`.

The output layer is specified via the node names in the node dictionary.

In [17]:
n_batch = 1
n_dirs = 5
n_neurons = 256
C = torch.randn(n_batch, n_dirs, n_neurons)

model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='crown')

(tensor([[ -3.5966,   3.4826,   0.1199,  -2.3320, -73.6734]],
        grad_fn=<ViewBackward0>),
 tensor([[ 13.9218, 105.1290,  19.2300,  36.2667,   3.3111]],
        grad_fn=<ViewBackward0>))

We can also use $\alpha$-CROWN to optimize the bounds of the directions.

In [18]:
model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='alpha-crown')

(tensor([[ -2.8655,  18.5538,   0.2205,   5.4271, -68.6798]]),
 tensor([[ 4.0163, 98.9213,  7.0998, 34.7713, -8.0158]]))

When using more iterations, the bounds may get slightly better.

In [19]:
model.bound_opts

{'conv_mode': 'patches',
 'sparse_intermediate_bounds': True,
 'sparse_conv_intermediate_bounds': True,
 'sparse_intermediate_bounds_with_ibp': True,
 'sparse_features_alpha': True,
 'sparse_spec_alpha': True,
 'minimum_sparsity': 0.9,
 'enable_opt_interm_bounds': False,
 'crown_batch_size': inf,
 'forward_refinement': False,
 'dynamic_forward': False,
 'forward_max_dim': 1000000000,
 'use_full_conv_alpha': True,
 'disabled_optimization': [],
 'use_full_conv_alpha_thresh': 512,
 'verbosity': 0,
 'optimize_graph': {'optimizer': None},
 'optimize_bound_args': {'enable_alpha_crown': True,
  'enable_beta_crown': False,
  'apply_output_constraints_to': None,
  'iteration': 20,
  'use_shared_alpha': False,
  'optimizer': 'adam',
  'keep_best': True,
  'fix_interm_bounds': True,
  'lr_alpha': 0.5,
  'lr_beta': 0.05,
  'lr_cut_beta': 0.005,
  'init_alpha': True,
  'lr_coeffs': 0.01,
  'intermediate_refinement_layers': [-1],
  'loss_reduction_func': <function auto_LiRPA.utils.<lambda>(x)>,
  's

In [20]:
def set_params(model, use_shared_alpha=False, iteration=20, early_stop_patience=10):
    model.bound_opts['optimize_bound_args']['use_shared_alpha'] = use_shared_alpha
    model.bound_opts['optimize_bound_args']['iteration'] = iteration
    model.bound_opts['optimize_bound_args']['early_stop_patience'] = early_stop_patience

In [21]:
set_params(model, iteration=100)
model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='alpha-crown')

(tensor([[ -2.8637,  18.6105,   0.2209,   5.4440, -68.6639]]),
 tensor([[ 3.9676, 98.8920,  7.0489, 34.7632, -8.0771]]))

# Converting Bounds to Points

CROWN only gives us **bounds** for linear combinations of neuron inputs that we specified in the matrix `C`. 
However, we need **points** - not the bounds.
Therefore, we also save the parameters of the backsubstituted inequalities and obtain the points in the input space that maximize/minimize the corresponding linear inequalities.
These maximizers/minimizers are then substituted into the inequalities for the neuron-inputs.

To obtain the coefficients of the inequalities, we need to set 
- `return_A = True` and also specify which coefficients we need by setting
- `needed_A_dict = {<layer_i> : [<layer_j>, <layer_k>]}` which will return the matrix of coefficients when substituting back from `layer_i` to `layer_j` and the matrix when substituting back from `layer_i` to `layer_k`

In [77]:
lbs11crown, ubs11crown, A_dict = model.compute_bounds(x=(x,), final_node_name='/input.11', method='alpha-crown', return_A=True, needed_A_dict={'/input.11': ['/0']})

The stored coefficients have shape `(batch, spec, *input_size)`. 

So if the input had shape `(1,1,1,784)` (which is `(batch, input_size1, input_size2, input_size3)`) and the matrix of specifications had `5` inequalities, the stored coefficients will have shape `(1, 5, 1, 1, 784)` (which is `(batch, spec, input_size1, input_size2, input_size3)`.

If no matrix of specifications is given, the `spec` dimension just has the shape of the layer (i.e. if it has 256 neurons, we have `spec = 256`)

In [78]:
lA_neurons = A_dict['/input.11']['/0']['lA']
lb_neurons = A_dict['/input.11']['/0']['lbias']

uA_neurons = A_dict['/input.11']['/0']['uA']
ub_neurons = A_dict['/input.11']['/0']['ubias']

print("A.shape = ", lA_neurons.shape)

A.shape =  torch.Size([1, 256, 1, 1, 784])


After obtaining the coefficients for the **neurons**, we now obtain the coefficients for the sampled **directions**.

In [62]:
lbs11crown, ubs11crown, A_dict = model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='alpha-crown', return_A=True, needed_A_dict={'/input.11': ['/0']})

In [64]:
lA = A_dict['/input.11']['/0']['lA']
lb = A_dict['/input.11']['/0']['lbias']

uA = A_dict['/input.11']['/0']['uA']
ub = A_dict['/input.11']['/0']['ubias']

print("A.shape = ", lA.shape)

A.shape =  torch.Size([1, 5, 1, 1, 784])


In [65]:
def flatten2matrix(A_tensor, batch_id=0):
    """
    Returns matrix of shape (spec, input_flat) corresponding to batch_id of the A_tensor.
    """
    # reshape to (batch, spec, input_flat), then take specific batch (0th batch)
    A_mat = A_tensor.reshape(A_tensor.shape[0], A_tensor.shape[1], -1)[batch_id, :]    
    return A_mat

def flatten2vector(b_tensor, batch_id=0):
    """
    Returns flat input vector corresponding to the specified batch.
    """
    b_vec = b_tensor.reshape(b_tensor.shape[0], -1)[batch_id, :]
    return b_vec

The following cell is just to demonstrate that we get the same bounds as for the specification, if we calculate them by hand.
(minor differences are expected due to randomization in the SGD procedure)

In [79]:
lA_mat = flatten2matrix(lA)
lb_vec = flatten2vector(lb)

uA_mat = flatten2matrix(uA)
ub_vec = flatten2vector(ub)

x_L_vec = flatten2vector(x.ptb.x_L)
x_U_vec = flatten2vector(x.ptb.x_U)

lA_neg = torch.minimum(torch.zeros(1), lA_mat)
lA_pos = torch.maximum(torch.zeros(1), lA_mat)

uA_neg = torch.minimum(torch.zeros(1), uA_mat)
uA_pos = torch.maximum(torch.zeros(1), uA_mat)

lo = lA_pos.matmul(x_L_vec) + lA_neg.matmul(x_U_vec) + lb_vec

print(lo)

tensor([ -2.8642,  18.5939,   0.2207,   5.4452, -68.6601])


Now get the inputs that are used to calculate the lower bound of the output.

For computation of lower bounds:
- if the coefficient of input $x_i$ is negative, we take the *upper* bound of $x_i$
- if the coefficient of input $x_i$ is positive, we take the *lower* bound of the input

to calculate the lower bound of the specification output

Vice-versa for computation of the upper bound on the specification output.

In [81]:
x_lo = x_L_vec * (lA_mat > 0) + x_U_vec * (lA_mat < 0)
x_up = x_U_vec * (uA_mat > 0) + x_L_vec * (uA_mat < 0)

Then calculate bounds of the neuron inputs using these obtained values of the $x_i$

In [82]:
lA_neurons_mat = flatten2matrix(lA_neurons)
lb_neurons_vec = flatten2vector(lb_neurons)

uA_neurons_mat = flatten2matrix(uA_neurons)
ub_neurons_vec = flatten2vector(ub_neurons)

In [83]:
l_neurons_subs = x_lo.matmul(lA_neurons_mat.T) + lb_neurons_vec
u_neurons_subs = x_lo.matmul(uA_neurons_mat.T) + ub_neurons_vec

In [84]:
l_neurons_subs

tensor([[-0.8823, -1.9052, -1.3219,  ..., -6.1794, -2.7905, -0.5578],
        [-0.8823, -1.9052, -1.3219,  ..., -6.1794, -2.7905, -0.5578],
        [-0.8823, -1.9052, -1.3219,  ..., -6.1794, -2.7905, -0.5578],
        [-0.8807, -1.8992, -1.3147,  ..., -6.1559, -2.7797, -0.5579],
        [-0.9500, -2.4437, -1.9653,  ..., -8.3334, -3.7501, -0.4112]])

In [85]:
u_neurons_subs

tensor([[-0.3018, -0.5380,  0.0490,  ..., -1.9711, -0.6990, -0.1242],
        [-0.3018, -0.5380,  0.0490,  ..., -1.9711, -0.6990, -0.1242],
        [-0.3018, -0.5380,  0.0490,  ..., -1.9711, -0.6990, -0.1242],
        [-0.3014, -0.5367,  0.0505,  ..., -1.9666, -0.6968, -0.1245],
        [-0.4520, -1.0317, -0.4799,  ..., -3.6531, -1.4987, -0.1012]])

In [101]:
C_mat = flatten2matrix(C)

# multiply each row of the specification (i.e. each direction) with each column of l_neurons_subs.T
# (i.e. the rows of l_neurons_subs, which are the minimizers of the respective directions).
torch.maximum(torch.zeros(1), C_mat).matmul(l_neurons_subs.T) + torch.minimum(torch.zeros(1), C_mat).matmul(u_neurons_subs.T)

tensor([[-154.4405, -154.4405, -154.4405, -153.7955, -165.0589],
        [-122.0603, -122.0603, -122.0603, -121.5373, -109.7246],
        [-171.8250, -171.8250, -171.8250, -171.0766, -185.8952],
        [-154.2048, -154.2048, -154.2048, -153.5711, -156.8310],
        [-200.7205, -200.7205, -200.7205, -199.8855, -230.7391]])

In [97]:
C_mat.shape

torch.Size([5, 256])

In [98]:
l_neurons_subs.shape

torch.Size([5, 256])

In [56]:
model.roots()[0].perturbation.concretize(x, A, sign=-1) + A_dict['/input.11']['/0']['lbias']

tensor([[ -4.4035, -44.7540, -81.8630, -27.9624,   2.5924]])

# Building LP Model

There is at least some code available to build LP and MILP models, but it doesn't seem to be maintained/is broken now. Maybe we can repair and use it.

In [89]:
model.build_solver_module(model_type='lp')

AttributeError: 'function' object has no attribute 'CONTINUOUS'

In [86]:
import gurobipy as grb

In [88]:
model.model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-27
